# 2. Macro VAE Training

Train and evaluate the Conditional VAE for macro scenario generation.

## Contents
1. Data Preparation
2. Model Architecture
3. Training
4. Evaluation & Generation

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from privatecredit.data import MacroScenarioGenerator
from privatecredit.models import MacroVAE
from privatecredit.models.macro_vae import MacroVAEConfig, MacroDataset, MacroVAETrainer

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

## 1. Data Preparation

In [ ]:
# Generate training data
macro_gen = MacroScenarioGenerator(n_months=60, start_date='2020-01-01')

n_paths = 100  # paths per scenario
all_data = []
all_labels = []

scenario_names = ['baseline', 'adverse', 'severely_adverse', 'stagflation']
numeric_cols = ['gdp_growth_yoy', 'unemployment_rate', 'inflation_rate',
                'policy_rate', 'yield_10y', 'credit_spread_ig',
                'credit_spread_hy', 'property_price_index', 'equity_return']

for idx, name in enumerate(scenario_names):
    for i in range(n_paths):
        macro_gen.rng = np.random.default_rng(42 + i + idx * 1000)
        df = macro_gen.generate_scenario(name)
        all_data.append(df[numeric_cols].values)
        all_labels.append(idx)

all_data = np.array(all_data)
all_labels = np.array(all_labels)
print(f"Data shape: {all_data.shape}")

In [ ]:
# Create datasets
dataset = MacroDataset(all_data, all_labels, normalize=True)

# Train/val split
n_train = int(0.8 * len(dataset))
train_idx = np.random.choice(len(dataset), n_train, replace=False)
val_idx = np.setdiff1d(np.arange(len(dataset)), train_idx)

train_data = torch.utils.data.Subset(dataset, train_idx)
val_data = torch.utils.data.Subset(dataset, val_idx)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)

## 2. Model Architecture

In [ ]:
# Configuration
config = MacroVAEConfig(
    n_macro_vars=9,
    seq_length=60,
    n_scenarios=4,
    hidden_dim=128,
    latent_dim=32,
    n_lstm_layers=2,
    dropout=0.2,
    n_epochs=50,
    batch_size=32,
    learning_rate=1e-3,
    kl_weight=0.1
)

model = MacroVAE(config)
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

## 3. Training

In [ ]:
trainer = MacroVAETrainer(model, config, device=device)
history = trainer.train(train_loader, val_loader, verbose=True)

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Val')
axes[0].set_title('Total Loss')
axes[0].legend()

axes[1].plot(history['train_recon'], label='Train')
axes[1].plot(history['val_recon'], label='Val')
axes[1].set_title('Reconstruction Loss')
axes[1].legend()

axes[2].plot(history['train_kl'], label='Train')
axes[2].plot(history['val_kl'], label='Val')
axes[2].set_title('KL Divergence')
axes[2].legend()

plt.tight_layout()
plt.show()

## 4. Evaluation & Generation

In [ ]:
# Generate samples from each scenario
model.eval()
generated = {}

for idx, name in enumerate(scenario_names):
    scenario_tensor = torch.tensor([idx], device=device)
    samples = model.generate(scenario_tensor, seq_length=60, n_samples=10)
    samples = samples.cpu().numpy()
    # Denormalize
    samples = samples * dataset.std.numpy() + dataset.mean.numpy()
    generated[name] = samples

print(f"Generated shape per scenario: {generated['baseline'].shape}")

In [ ]:
# Plot generated vs real scenarios
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
var_names = ['GDP Growth', 'Unemployment', 'Inflation', 'HY Spread']
var_idx = [0, 1, 2, 6]

for ax, name, v_idx in zip(axes.flat, var_names, var_idx):
    for sc_name, samples in generated.items():
        for i in range(min(5, len(samples))):
            ax.plot(samples[i, :, v_idx], alpha=0.5, label=sc_name if i == 0 else None)
    ax.set_title(name)
    ax.set_xlabel('Month')
    ax.legend()

plt.tight_layout()
plt.show()

## Summary

- Trained Conditional VAE on 400 macro scenarios (4 types x 100 paths)
- Model captures scenario-specific dynamics
- Can generate new scenarios conditioned on scenario type

**Next:** Compare with GAN and Flow models (Notebook 08)